In [1]:
#!pip3 install --user cupy-cuda102

In [0]:
import cupy as cp
import numpy as np

Función gradiente:

In [0]:
def gfo_cp_mark(Sigma,x):
    '''
    gradiente de la función objetivo 1/2*x.t*Sigma*x
    input: matriz Sigma y vector x
    output: producto matriz-vector Sigma*x
    '''
    first_block = Sigma@x
    return first_block

Función Hessiana:

In [0]:
def Hfo_cp_mark(Sigma):
    '''
    Hessiana de la función objetivo 1/2*x.t*Sigma*x
    input: matriz Sigma
    output: matriz Sigma
    '''
    first_block = Sigma 
    return first_block

In [5]:
import pandas as pd
import fix_yahoo_finance as yf
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import time

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
stocks = ['COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
'UPS','7203.T','PEP','MRK','1398.HK','MA','T']

In [0]:

def extraer_datos_yahoo(stocks, start='2015-01-01', end='2020-04-30'):
    '''
    Descarga la información de precios de acciones al cierre para un periodo determinado
    Inputs:
        stocks - lista de acciones a descargar, las 50 especificadas previamente
        'COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
        'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
        'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
        'UPS','7203.T','PEP','MRK','1398.HK','MA','T'
        start - fecha inicio a partir de la cual se requieren los precios de las acciones, formato 'YYYY-MM-DD'
        end - fecha final hasta donde se requieren los precios de las acciones, formato 'YYYY-MM-DD'
    Output:
        datos - dataframe con 50 columnas, una por acción, tantos renglones como días hábiles del periodo
        solicitado (1340 en el caso de las fechas default), cada celda es el precio de una acción al cierre, 
        de un día específico
    '''
    df_c = yf.download(stocks, start, end).Close
    base = df_c['AAPL'].dropna().to_frame()
    for i in range(0,50):
        base = base.join(df_c.iloc[:,i].to_frame(), lsuffix='_caller', rsuffix='_other')
    base = base.drop(columns=['AAPL_caller'])
    base = base.rename(columns={"AAPL_other": "AAPL"})
    base = base.fillna(method='ffill')
    base = base.fillna(method='bfill')
    return base

In [9]:
datos = extraer_datos_yahoo(stocks)

[*********************100%***********************]  50 of 50 downloaded


In [0]:
def calcular_rendimiento_vector(x):
  """
  Función para calcular el rendimiento esperado

  params:
      x     vector de precios
  
  return:
      r_est rendimiento esperado diario
  """

  # Definimos precios iniciales y finales como arreglo alojado en la gpu
  x_o = cp.asarray(x)
  x_f = x_o[1:]

  # Calculamos los rendimientos diarios
  r = cp.log(x_f/x_o[:-1])

  return r

In [0]:
def calcular_rendimiento(X):
  """
  Función para calcular el rendimiento esperado para un conjunto de acciones

  params:
      X      matriz mxn de precios, donde:
             m es el número de observaciones y
             n el número de acciones
  
  return:
      r_est rvector de rendimientos esperados
  """
  m,n = X.shape
  r_est = cp.zeros(n)
  X = cp.asarray(X)

  for i in range(n):
    r_est[i] = calcular_rendimiento_vector(X[:,i]).mean()

  return 264*r_est

In [0]:
def calcular_varianza(X):

  """
  Función para calcular el la matriz de varianzas y covarianzas para un conjunto de acciones

  params:
      X      matriz mxn de precios, donde:
             m es el número de observaciones y
               n el número de acciones
  
  return:
      S  matriz de varianzas y covarianzas
  """
  m,n=X.shape
  X = cp.asarray(X)

  X_m = cp.zeros((m-1,n))

  for i in range(n):
    X_m[:,i] = calcular_rendimiento_vector(X[:,i]) - calcular_rendimiento_vector(X[:,i]).mean()

  S = (cp.transpose(X_m)@X_m)/(m-2)

  return S

In [0]:
Sigma = calcular_varianza(datos)

In [ ]:
x = cp.ones(50)

In [ ]:
gfo_cp_mark(Sigma,x)

In [ ]:
Hfo_cp_mark(Sigma)